In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import asyncio
from requests_html import AsyncHTMLSession
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
  options.add_argument('E:/Programs/geckodriver.exe')
  options.add_argument('--headless')
  options.add_argument("--start-with-cache")
  options.add_argument("0")
  options.add_argument("--disk-cache-size=0")
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_parquet(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_parquet(filename, index=False)
    else:
        data.to_parquet(filename, index=False)

In [3]:
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [7]:
save = 0
strng = 0
pg = 0

In [6]:
pageinfoserch = 1
rangesal = ['3to6', '6to10', '10to15', '15to25', '25to50', '50to75', '75to100', '100to500', '0to3']
for filt in range(strng,len((rangesal))):
    strng = filt 
    if pageinfoserch == 1:
        driver.get('https://www.naukri.com/jobs-in-india-'+str(pg)+'?ctcFilter='+rangesal[filt])
        pageinfoserch = 0
    else:
        driver.get('https://www.naukri.com/jobs-in-india?ctcFilter='+rangesal[filt])
    time.sleep(2)
    total = int(driver.find_element(By.CLASS_NAME, 'styles_h1-wrapper__mHVA1').text.split('of')[-1].split('\n')[0].strip())
    ################################
    sub_save = 0
    ################################
    while True:    
        time.sleep(3)
        try:
            next_button = driver.find_element(By.LINK_TEXT, "Next")
        except NoSuchElementException:
            print("...")
            driver.quit()
            driver = driver_setup()
            driver.get('https://www.naukri.com/jobs-in-india-'+str(pg)+'?ctcFilter='+rangesal[filt])
            time.sleep(3)
        
        job_articles = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'cust-job-tuple')))
        for job_element in job_articles:
            data = {}
            data['sal_range'] = filt 
            try:
                data['link'] = job_element.find_element(By.TAG_NAME, 'a').get_attribute('href')
            except:
                continue
            if data != {}:
                try:
                    print(f'\rpg: {pg} / {total//20}, Progress: {save} filtlevel_save: {sub_save}, total: {total}, filtlevel: {filt}', end='')
                except:
                    print(f'\rProgress: {save}, filtlevel_save: {sub_save}, total: {total}, filtlevel: {filt}', end='')
                save += 1
                sub_save +=1
                tempfilename = 'links.parquet'  # Change the file extension to '.parquet'
                save_data_to_disk(pd.DataFrame(data, index=[0]), tempfilename)
                tempdf = pd.DataFrame()
                # linksdf = pd.concat([linksdf,pd.DataFrame(data, index=[0])], ignore_index=True)
        time.sleep(1)
        next_button = driver.find_element(By.LINK_TEXT, "Next")
        is_disabled = next_button.get_attribute('disabled') is not None
        if is_disabled or sub_save > 10000:
            break
        else:
            try:
                next_button.click()
            except:
                driver.refresh()
                time.sleep(2)
                print('*')
                continue                    
        pg = driver.current_url.split('india-')
        if len(pg) > 1:
            pg = (pg[1].split('?')[0])
        else:
            pg = 1
        print(end='')

pg: 67 / 128, Progress: 68653 filtlevel_save: 2061, total: 2576, filtlevel: 5

KeyboardInterrupt: 